In [ ]:
import os
import asyncio
import nest_asyncio
from openai import AsyncOpenAI

NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")

### 合成データ生成用プロンプトの設定

In [ ]:
client = AsyncOpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=NVIDIA_API_KEY,  # 取得したAPIキーを入力してください
)

TOPIC_GENERATION_PROMPT_TEMPLATE = """\
トピックが与えられた場合、そのトピックに関連する {n_subtopics} のサブトピックのリストを生成してください。
トピックは：{topic}
リストは番号なしで、サブトピックの説明なしでなければなりません。サブトピックはコンマで区切られる必要があります。リスト以外のテキストは存在してはなりません。
"""

QUESTION_PROMPT_TEMPLATE = """\
トピックが与えられた場合、そのトピックに関して{n_questions}個の質問を生成してください。
トピックは：{sub_topic}
リスト形式で、質問は改行文字で区切られる必要があります。リスト以外のテキストは存在してはなりません。
"""

RESPONSE_PROMPT_TEMPLATE = """\
質問が与えられた場合、その質問に対して考えられる2つの回答を生成してください。
質問は：{question}
リスト形式は以下の形式である必要があります：

RESPONSE A: ここに回答Aのテキストを入力
RESPONSE B: ここに回答Bのテキストを入力
"""

### トピックを設定し、サブトピックを生成

In [ ]:
# generate sub topics
async def generate_subtopics(client, topic, n_subtopics):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(
        topic=topic, n_subtopics=n_subtopics
    )
    response = await client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    return response


n_subtopics = 10
n_questions = 10
topic = "NVIDIA"

subtopics = await generate_subtopics(client, topic, n_subtopics)
subtopic_list = subtopics.choices[0].message.content.split(",")
print(subtopic_list)

['グラフィックカード、人工知能、ディープラーニング、ゲーミング、GPU、CUDA、TensorFlow、NVIDIA Shield、オートモーティブ、プロフェッショナルビジュアライゼーション']


### サブトピックに関する質問文を生成

In [35]:
# generate questions of sub topics
async def generate_questions(client, sub_topic, n_questions):
    prompt = QUESTION_PROMPT_TEMPLATE.format(
        sub_topic=sub_topic, n_questions=n_questions
    )
    response = await client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    if hasattr(response, "choices") and response.choices:
        return response.choices[0].message.content
    else:
        print(f"Unexpected response structure: {response}")
        return None


async def question_generator(client, subtopic_list, n_question):
    tasks = [
        generate_questions(client, subtopic, n_question) for subtopic in subtopic_list
    ]
    question_list = await asyncio.gather(*tasks)
    return question_list


nest_asyncio.apply()
question_list = asyncio.run(question_generator(client, subtopic_list, n_questions))
print(question_list)

# format questions
question_list_formatted = []
for question_set in question_list:
    question_list_formatted += question_set.split("\n\n")

['グラフィックカードの主な機能は何か？\n\n人工知能とディープラーニングの違いは何か？\n\nゲーミングに最適なGPUは何か？\n\nCUDAとは何か？\n\nTensorFlowはどのような用途に使用されるか？\n\nNVIDIA Shieldの主な機能は何か？\n\nオートモーティブ分野でグラフィックカードはどのように活用されるか？\n\nプロフェッショナルビジュアライゼーションに最適なGPUは何か？\n\nグラフィックカードの性能を向上させる方法は何か？\n\nディープラーニングとGPUの関係は何か？']


### 質問文に対して回答文を生成

In [ ]:
# generate response of each question
async def generate_responses(client, question):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question)
    response = await client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    if hasattr(response, "choices") and response.choices:
        return response.choices[0].message.content
    else:
        print(f"Unexpected response structure: {response}")
        return None


async def response_generator(client, question_list):
    tasks = [generate_responses(client, question) for question in question_list]
    response_list = await asyncio.gather(*tasks)
    return response_list


question_response_list = asyncio.run(
    response_generator(client, question_list_formatted)
)


print(question_response_list)

['RESPONSE A: グラフィックカードの主な機能は、コンピュータの画面に画像を表示することです。グラフィックカードは、コンピュータのCPUから画像データを受け取り、画像を処理して画面に表示します。\n\nRESPONSE B: グラフィックカードの主な機能は、コンピュータのゲームやグラフィック処理を高速化することです。グラフィックカードは、専用のGPU（Graphics Processing Unit）を搭載しており、グラフィック処理を高速化するために設計されています。', 'RESPONSE A: 人工知能（AI）とディープラーニング（DL）は、関連しているが異なる概念です。人工知能は、人間の知能を模倣するコンピューターシステムの開発を目指す広範な分野であり、機械学習、自然言語処理、ロボット工学などを含みます。一方、ディープラーニングは、人工知能の一部であり、ニューラルネットワークを使用してデータから学習し、特徴を抽出する手法です。ディープラーニングは、画像認識、音声認識、自然言語処理などのタスクに特に有効です。\n\nRESPONSE B: 人工知能とディープラーニングの主な違いは、範囲とアプローチにあります。人工知能は、人間の知能をコンピューターで再現することを目指す包括的な分野であり、推論、問題解決、学習などを含みます。ディープラーニングは、人工知能の一部であり、複数の層を持つニューラルネットワークを使用して、データから複雑なパターンを学習し、特徴を抽出します。ディープラーニングは、特に大量のデータを必要とするタスクに適しており、画像認識、音声認識、自然言語処理などの分野で広く使用されています。', 'RESPONSE A: NVIDIA GeForce RTX 3080は、ゲーミングに最適なGPUです。高性能、高速なフレームレート、そしてAIを活用したグラフィックス処理機能を備えています。\n\nRESPONSE B: AMD Radeon RX 6800 XTは、ゲーミングに最適なGPUです。高性能、高速なフレームレート、そして多くのゲームでNVIDIA GeForce RTX 3080と同等のパフォーマンスを提供します。', 'RESPONSE A: CUDAは、NVIDIAが開発した並列コンピューティングプラットフォームおよびプログラミングモ

### 質問文と回答文をまとめたリストを作成

In [ ]:
question_response_pair_list = []
for question, response_set in zip(question_list_formatted, question_response_list):
    question_response_pair_list.append(
        {
            "question": question,
            "responses": {
                "response_a": {
                    "response": response_set.split("RESPONSE B:")[0]
                    .replace("RESPONSE A:", "")
                    .strip()
                    .split("\n\n")[-1]
                    .strip()
                },
                "response_b": {
                    "response": response_set.split("RESPONSE B:")[-1]
                    .split("\n\n")[0]
                    .strip()
                },
            },
        }
    )

### 質問文と回答文から回答の性能評価

In [39]:
# running reward scoring model to evaluate the responses
def get_scores_from_response(openai_response_template):
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict


async def get_response_and_scores(client, question, response_content):
    messages = [
        {"role": "user", "content": question},
        {"role": "assistant", "content": response_content},
    ]
    response = await client.chat.completions.create(
        model="nvidia/nemotron-4-340b-reward",
        messages=messages,
    )
    scores = get_scores_from_response(response)
    return scores


# scoring for question:response pair set
async def process_question_response_pairs(client, question_response_score_list):
    tasks = []
    for question_response_pair in question_response_score_list:
        question = question_response_pair["question"]

        task_a = get_response_and_scores(
            client,
            question,
            question_response_pair["responses"]["response_a"]["response"],
        )
        task_b = get_response_and_scores(
            client,
            question,
            question_response_pair["responses"]["response_b"]["response"],
        )

        tasks.append((task_a, question_response_pair, "response_a"))
        tasks.append((task_b, question_response_pair, "response_b"))
    results = await asyncio.gather(*[task[0] for task in tasks])

    for i, (result, task_info) in enumerate(zip(results, tasks)):
        _, question_response_pair, response_key = task_info
        question_response_pair["responses"][response_key].update(result)


question_response_score_list = question_response_pair_list.copy()
await process_question_response_pairs(client, question_response_score_list)
print(question_response_score_list)

[{'question': 'グラフィックカードの主な機能は何か？', 'responses': {'response_a': {'response': 'グラフィックカードの主な機能は、コンピュータの画面に画像を表示することです。グラフィックカードは、コンピュータのCPUから画像データを受け取り、画像を処理して画面に表示します。', 'helpfulness': 2.796875, 'correctness': 3.03125, 'coherence': 3.8125, 'complexity': 1.3046875, 'verbosity': 1.125}, 'response_b': {'response': 'グラフィックカードの主な機能は、コンピュータのゲームやグラフィック処理を高速化することです。グラフィックカードは、専用のGPU（Graphics Processing Unit）を搭載しており、グラフィック処理を高速化するために設計されています。', 'helpfulness': 2.828125, 'correctness': 3.09375, 'coherence': 3.890625, 'complexity': 1.546875, 'verbosity': 1.328125}}}, {'question': '人工知能とディープラーニングの違いは何か？', 'responses': {'response_a': {'response': '人工知能（AI）とディープラーニング（DL）は、関連しているが異なる概念です。人工知能は、人間の知能を模倣するコンピューターシステムの開発を目指す広範な分野であり、機械学習、自然言語処理、ロボット工学などを含みます。一方、ディープラーニングは、人工知能の一部であり、ニューラルネットワークを使用してデータから学習し、特徴を抽出する手法です。ディープラーニングは、画像認識、音声認識、自然言語処理などのタスクに特に有効です。', 'helpfulness': 3.4375, 'correctness': 3.59375, 'coherence': 3.984375, 'complexity': 1.6796875, 'verbosity': 1.796875}, 'response_b': {'response':

In [41]:
question_response_pair_list

[{'question': 'グラフィックカードの主な機能は何か？',
  'responses': {'response_a': {'response': 'グラフィックカードの主な機能は、コンピュータの画面に画像を表示することです。グラフィックカードは、コンピュータのCPUから画像データを受け取り、画像を処理して画面に表示します。'},
   'response_b': {'response': 'グラフィックカードの主な機能は、コンピュータのゲームやグラフィック処理を高速化することです。グラフィックカードは、専用のGPU（Graphics Processing Unit）を搭載しており、グラフィック処理を高速化するために設計されています。'}}},
 {'question': '人工知能とディープラーニングの違いは何か？',
  'responses': {'response_a': {'response': '人工知能（AI）とディープラーニング（DL）は、関連しているが異なる概念です。人工知能は、人間の知能を模倣するコンピューターシステムの開発を目指す広範な分野であり、機械学習、自然言語処理、ロボット工学などを含みます。一方、ディープラーニングは、人工知能の一部であり、ニューラルネットワークを使用してデータから学習し、特徴を抽出する手法です。ディープラーニングは、画像認識、音声認識、自然言語処理などのタスクに特に有効です。'},
   'response_b': {'response': '人工知能とディープラーニングの主な違いは、範囲とアプローチにあります。人工知能は、人間の知能をコンピューターで再現することを目指す包括的な分野であり、推論、問題解決、学習などを含みます。ディープラーニングは、人工知能の一部であり、複数の層を持つニューラルネットワークを使用して、データから複雑なパターンを学習し、特徴を抽出します。ディープラーニングは、特に大量のデータを必要とするタスクに適しており、画像認識、音声認識、自然言語処理などの分野で広く使用されています。'}}},
 {'question': 'ゲーミングに最適なGPUは何か？',
  'responses': {'response_a': {'response': 'NVIDIA GeForce RTX 308